<a href="https://colab.research.google.com/github/chinox02/TugasDLTM4_Yudharso-Pala/blob/main/Klasifikasi_Dari_Kodingan_Kaggle_Yudharso_pala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from tqdm import tqdm
from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import glob
import pandas as pan
import matplotlib.pyplot as plotter
import warnings
warnings.filterwarnings('ignore')

In [ ]:
image_data='/content/drive/MyDrive/datasetTM4deeplearning/dataset'
pd.DataFrame(os.listdir(image_data),columns=['Files_Name'])

In [ ]:
files = [i for i in glob.glob(image_data + "//*//*")]
np.random.shuffle(files)
labels = [os.path.dirname(i).split("/")[-1] for i in files]
data = zip(files, labels)
dataframe = pan.DataFrame(data, columns = ["Image", "Label"])
dataframe

In [ ]:
files = [i for i in glob.glob(image_data + "//*//*")]
np.random.shuffle(files)
labels = [os.path.dirname(i).split("/")[-1] for i in files]
data = zip(files, labels)
dataframe = pan.DataFrame(data, columns = ["Image", "Label"])
dataframe

In [ ]:
sns.countplot(x = dataframe["Label"])
plotter.xticks(rotation = 0);

In [ ]:
train_data_dir =image_data
batch_size = 64
target_size = (224,224)
validation_split = 0.2
train= tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    validation_split=validation_split,
    subset="training",
    seed=100,
    image_size=target_size,
    batch_size=batch_size,
)
validation= tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    validation_split=validation_split,
    subset="validation",
    seed=200,
    image_size=target_size,
    batch_size=batch_size,
)

In [ ]:
train_size = len(train)
val_size = len(validation)
total = train_size + val_size

print(f'train samples count:\t\t{train_size}\t({(100 * train_size/total):.2f}%)')
print(f'validation samples count:\t{val_size}\t({(100 * val_size/total):.2f}%)')
print('================================================')
print(f'Total:\t\t\t\t{total}\t({(100 * total/total):.2f}%)')

In [ ]:
class_names=train.class_names
class_names

In [ ]:
plt.figure(figsize=(15, 20))
for images, labels in train.take(1):
    for i in range(12):
        ax = plt.subplot(12, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
base_model = tf.keras.applications.EfficientNetV2M(input_shape=(224,224,3),include_top=False,weights='imagenet')
base_model.trainable = False
keras_model=keras.models.Sequential()
keras_model.add(base_model)
keras_model.add(keras.layers.Flatten())
keras_model.add(keras.layers.Dropout(0.5))
keras_model.add(keras.layers.Dense(6,activation=tf.nn.softmax))
keras_model.summary()

In [ ]:
tf.keras.utils.plot_model(keras_model, to_file='model.png', show_shapes=True, show_layer_names=True,show_dtype=True,dpi=80)

In [ ]:
checkpoint =ModelCheckpoint("my_keras_model.h5", save_best_only=True)

In [ ]:
early_stopping =EarlyStopping(patience=5, restore_best_weights=True)

In [35]:
keras_model.compile(optimizer ='Adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])
hist=keras_model.fit_generator(train,epochs=100,validation_data=validation,callbacks=[checkpoint,early_stopping])

Epoch 1/100
8/8 [==============================] - 254s 26s/step - loss: 0.2266 - accuracy: 0.9417 - val_loss: 0.6485 - val_accuracy: 0.8750
Epoch 2/100
8/8 [==============================] - 194s 24s/step - loss: 0.2048 - accuracy: 0.9333 - val_loss: 0.6354 - val_accuracy: 0.8583
Epoch 3/100
8/8 [==============================] - 197s 25s/step - loss: 0.1124 - accuracy: 0.9563 - val_loss: 0.4530 - val_accuracy: 0.9083
Epoch 4/100
8/8 [==============================] - 196s 25s/step - loss: 0.0837 - accuracy: 0.9667 - val_loss: 0.4035 - val_accuracy: 0.9167
Epoch 5/100
8/8 [==============================] - 196s 25s/step - loss: 0.0945 - accuracy: 0.9688 - val_loss: 0.6742 - val_accuracy: 0.8917
Epoch 6/100
8/8 [==============================] - 200s 25s/step - loss: 0.1127 - accuracy: 0.9646 - val_loss: 0.4904 - val_accuracy: 0.9167
Epoch 7/100
8/8 [==============================] - 201s 25s/step - loss: 0.0905 - accuracy: 0.9646 - val_loss: 0.5958 - val_accuracy: 0.9083
Epoch 8/100
8

In [36]:
score, acc = keras_model.evaluate(validation)
print('Test Loss =', score)
print('Test Accuracy =', acc)

2/2 [==============================] - 38s 17s/step - loss: 0.4035 - accuracy: 0.9167
Test Loss = 0.4035080671310425
Test Accuracy = 0.9166666865348816


In [ ]:
hist_=pd.DataFrame(hist.history)
hist_

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(hist_['loss'],label='Train_Loss')
plt.plot(hist_['val_loss'],label='Validation_Loss')
plt.title('Train_Loss & Validation_Loss',fontsize=20)
plt.legend()
plt.subplot(1,2,2)
plt.plot(hist_['accuracy'],label='Train_Accuracy')
plt.plot(hist_['val_accuracy'],label='Validation_Accuracy')
plt.title('Train_Accuracy & Validation_Accuracy',fontsize=20)
plt.legend()